In [29]:
import numpy as np
import cv2
import math

def equirectangular_to_sphere_points(panorama, radius=1.0, density=1000):
    height, width = panorama.shape[:2]
    points = []
    colors = []
    
    for i in range(0, height, height//density):
        for j in range(0, width, width//density):
            theta = (i / height) * math.pi
            phi = (j / width) * 2 * math.pi
            
            x = radius * math.sin(theta) * math.cos(phi)
            y = radius * math.sin(theta) * math.sin(phi)
            z = radius * math.cos(theta)
            
            color = panorama[i, j]
            
            points.append([x, y, z])
            colors.append(color)
    
    return np.array(points), np.array(colors)

def shift_pov(points, shift):
    return points - shift

def sphere_points_to_equirectangular(points, colors, output_size):
    height, width = output_size
    panorama = np.zeros((height, width, 3), dtype=np.uint8)
    
    for point, color in zip(points, colors):
        x, y, z = point
        theta = math.acos(z / np.linalg.norm(point))
        phi = math.atan2(y, x)
        
        if phi < 0:
            phi += 2 * math.pi
        
        i = int((theta / math.pi) * height) - 1
        j = int((phi / (2 * math.pi)) * width) - 1
        
        panorama[i, j] = color
    
    # Fill in gaps
    panorama = cv2.dilate(panorama, np.ones((5,5), np.uint8))
    panorama = cv2.medianBlur(panorama, 5)
    
    return panorama

def shift_pov_from_pano(input_path, output_path, x=0, y=0, z=0):
    # Load the panorama image
    panorama = cv2.imread(input_path)
    height, width = panorama.shape[:2]

    # Create point cloud
    points, colors = equirectangular_to_sphere_points(panorama, radius=1.0, density=1000)

    # Shift POV
    shift = np.array([x, y, z])
    shifted_points = shift_pov(points, shift)

    # Generate new panorama from shifted points
    new_panorama = sphere_points_to_equirectangular(shifted_points, colors, (height, width))

    # Save or display results
    cv2.imwrite(output_path, new_panorama)

In [30]:
import os

os.makedirs("data/raw/traditional_cv/exploration", exist_ok=True)
# parameter exloration (x, y, z)
for y in range(1, 10, 1):
    y = y / 10
    for z in range(-1, -5, -1):
        z = z / 10
        img_1 = "data/raw/road_shoulder/gsv/image/gsv_panorama/_8kvJmLvnIicc28eN1JsKQ.jpg"
        img_2 = "data/raw/road_shoulder/gsv/image/gsv_panorama/_b9eQrq8OXPZLm_vUMabgg.jpg"
        img_3 = "data/raw/road_shoulder/gsv/image/gsv_panorama/_cirws0yB8TRyuJ9gpwrYg.jpg"
        for img in [img_1, img_2, img_3]:
            if not os.path.exists(f"data/raw/traditional_cv/exploration/y_{y}_z_{z}_{img.split('/')[-1]}"):
                print(f"y: {y}, z: {z}, img: {img.split('/')[-1]}")
                shift_pov_from_pano(img, f"data/raw/traditional_cv/exploration/y_{y}_z_{z}_{img.split('/')[-1]}", y=y, z=z)

y: 0.5, z: -0.1, img: _8kvJmLvnIicc28eN1JsKQ.jpg


y: 0.5, z: -0.1, img: _b9eQrq8OXPZLm_vUMabgg.jpg
y: 0.5, z: -0.1, img: _cirws0yB8TRyuJ9gpwrYg.jpg
y: 0.5, z: -0.2, img: _8kvJmLvnIicc28eN1JsKQ.jpg
y: 0.5, z: -0.2, img: _b9eQrq8OXPZLm_vUMabgg.jpg
y: 0.5, z: -0.2, img: _cirws0yB8TRyuJ9gpwrYg.jpg
y: 0.5, z: -0.3, img: _8kvJmLvnIicc28eN1JsKQ.jpg
y: 0.5, z: -0.3, img: _b9eQrq8OXPZLm_vUMabgg.jpg
y: 0.5, z: -0.3, img: _cirws0yB8TRyuJ9gpwrYg.jpg
y: 0.5, z: -0.4, img: _8kvJmLvnIicc28eN1JsKQ.jpg
y: 0.5, z: -0.4, img: _b9eQrq8OXPZLm_vUMabgg.jpg
y: 0.5, z: -0.4, img: _cirws0yB8TRyuJ9gpwrYg.jpg
y: 0.6, z: -0.1, img: _8kvJmLvnIicc28eN1JsKQ.jpg
y: 0.6, z: -0.1, img: _b9eQrq8OXPZLm_vUMabgg.jpg
y: 0.6, z: -0.1, img: _cirws0yB8TRyuJ9gpwrYg.jpg
y: 0.6, z: -0.2, img: _8kvJmLvnIicc28eN1JsKQ.jpg
y: 0.6, z: -0.2, img: _b9eQrq8OXPZLm_vUMabgg.jpg
y: 0.6, z: -0.2, img: _cirws0yB8TRyuJ9gpwrYg.jpg
y: 0.6, z: -0.3, img: _8kvJmLvnIicc28eN1JsKQ.jpg
y: 0.6, z: -0.3, img: _b9eQrq8OXPZLm_vUMabgg.jpg
y: 0.6, z: -0.3, img: _cirws0yB8TRyuJ9gpwrYg.jpg
y: 0.6, z: -0.4, img

In [2]:
import pandas as pd
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# Make output directory
os.makedirs("data/raw/traditional_cv/test", exist_ok=True)

# Get test images
test_image_list = os.listdir("data/processed/road_shoulder/cyclegan_filtered/testA")

# Read CSV
id_df = pd.read_csv("data/raw/road_shoulder/gsv/metadata/gsv_metadata_cv_filtered.csv")
id_df["mly_id"] = id_df["mly_id"].astype(int).astype(str).apply(lambda x: x + ".jpg")

# Filter id_df with test_image_list based on mly_id
id_df = id_df[id_df["mly_id"].isin(test_image_list)]
panoid_list = id_df["panoid"].values.tolist()

# Loop through data/raw/road_shoulder/gsv/image/gsv_panorama and shift POV by 0.8, -0.3
image_list = os.listdir("data/raw/road_shoulder/gsv/image/gsv_panorama")

def process_image(image):
    if image.replace(".jpg", "") not in panoid_list:
        return
    shift_pov_from_pano(
        f"data/raw/road_shoulder/gsv/image/gsv_panorama/{image}",
        f"data/raw/traditional_cv/test/{image}",
        y=0.8, z=-0.3
    )

# Use ThreadPoolExecutor to parallelize the processing with a progress bar using as_completed
with ThreadPoolExecutor() as executor:
    futures = {executor.submit(process_image, image): image for image in image_list}
    for future in tqdm(as_completed(futures), total=len(futures)):
        # You can handle exceptions here if needed
        try:
            future.result()
        except Exception as e:
            print(f"Error processing image: {e}")

 25%|██▌       | 4200/16611 [16:27<17:37, 11.74it/s]  

Error processing image: range() arg 3 must not be zero


 27%|██▋       | 4513/16611 [17:10<27:54,  7.22it/s]

Error processing image: range() arg 3 must not be zero


 31%|███       | 5135/16611 [21:47<53:15,  3.59it/s]  

...

 94%|█████████▎| 15547/16611 [1:21:21<07:44,  2.29it/s]

Error processing image: range() arg 3 must not be zero


 99%|█████████▉| 16427/16611 [1:26:17<00:43,  4.28it/s]

Error processing image: range() arg 3 must not be zero


100%|██████████| 16611/16611 [1:30:17<00:00,  3.07it/s]


In [3]:
from zensvi.cv import Segmenter
import os 

os.makedirs('data/processed/traditional_cv/segmented_images', exist_ok=True)
os.makedirs('data/processed/traditional_cv/segmented_labels', exist_ok=True)
segmenter = Segmenter()
segmenter.segment('data/raw/traditional_cv/test',
                  dir_image_output='data/processed/traditional_cv/segmented_images',
                  dir_summary_output='data/processed/traditional_cv/segmented_labels',
                  save_image_options='blend_image',
                  save_format='csv',
                  csv_format='wide')

Using GPU


preprocessor_config.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

/data/koichi/miniconda3/envs/gan_svi/lib/python3.10/site-packages/transformers/utils/deprecation.py:165: UserWarning: The following named arguments are not valid for `Mask2FormerImageProcessor.__init__` and were ignored: '_max_size'
  return func(*args, **kwargs)


config.json:   0%|          | 0.00/77.3k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/190M [00:00<?, ?B/s]

Processing outer batches of size 498:   0%|          | 0/1 [00:00<?, ?it/s]

Processing outer batch #1:   0%|          | 0/498 [00:00<?, ?it/s]

In [12]:
# transform panorama to perspectives
from zensvi.transform import ImageTransformer
import os
os.makedirs("data/processed/traditional_cv/perspective_images/all", exist_ok=True)
transformer = ImageTransformer(dir_input="data/raw/traditional_cv/test",
                               dir_output="data/processed/traditional_cv/perspective_images/all")
transformer.transform_images(style_list="perspective")

# copy files with "Direction_0" to "data/processed/traditional_cv/perspective_images/Direction_0"
os.makedirs("data/processed/traditional_cv/perspective_images/Direction_0", exist_ok=True)
for image in os.listdir("data/processed/traditional_cv/perspective_images/all/perspective"):
    if "Direction_0" in image:
        os.rename(f"data/processed/traditional_cv/perspective_images/all/perspective/{image}", 
                  f"data/processed/traditional_cv/perspective_images/Direction_0/{image}")
        
# run segmentation on perspective images
os.makedirs('data/processed/traditional_cv/segmented_images/Direction_0', exist_ok=True)
os.makedirs('data/processed/traditional_cv/segmented_labels/Direction_0', exist_ok=True)
segmenter = Segmenter()
segmenter.segment('data/processed/traditional_cv/perspective_images/Direction_0',
                  dir_image_output='data/processed/traditional_cv/segmented_images/Direction_0',
                  dir_summary_output='data/processed/traditional_cv/segmented_labels/Direction_0',
                  save_image_options='blend_image',
                  save_format='csv',
                  csv_format='wide')

Converting to perspective: 100%|██████████| 498/498 [00:00<00:00, 1574.20it/s]

Using GPU


Processing outer batches of size 498:   0%|          | 0/1 [00:00<?, ?it/s]

Processing outer batch #1:   0%|          | 0/498 [00:00<?, ?it/s]

In [11]:
import pandas as pd
ground_truth = pd.read_csv("data/processed/road_shoulder/panorama/test/B_segmentation/pixel_ratios.csv")
# rename vegetation to ground_truth_vegetation
ground_truth.rename(columns={"vegetation": "ground_truth_vegetation",
                             "building": "ground_truth_building",
                             "sky": "ground_truth_sky",
                             }, inplace=True)
ground_truth["filename_key"] = ground_truth["filename_key"].astype(str)
sidewalk = pd.read_csv("data/processed/sidewalk/panorama/test/B_segmentation/pixel_ratios.csv")
# rename vegetation to ground_truth_vegetation
sidewalk.rename(columns={"vegetation": "ground_truth_sidewalk_vegetation",
                         "building": "ground_truth_sidewalk_building",
                            "sky": "ground_truth_sidewalk_sky",
                            }, inplace=True)
sidewalk["filename_key"] = sidewalk["filename_key"].astype(str)
shifted_pano = pd.read_csv("data/processed/traditional_cv/segmented_labels/pixel_ratios.csv")
# rename vegetation to predicted_vegetation
shifted_pano.rename(columns={"vegetation": "shifted_pano_vegetation",
                                "building": "shifted_pano_building",
                                "sky": "shifted_pano_sky",
                                }, inplace=True)
shifted_per = pd.read_csv("data/processed/traditional_cv/segmented_labels/Direction_0/pixel_ratios.csv")
shifted_per["filename_key"] = shifted_per["filename_key"].astype(str).apply(lambda x: x.replace("_Direction_0_FOV_90_aspect_9--16_raw", ""))
shifted_per.rename(columns={"vegetation": "shifted_per_vegetation",
                                "building": "shifted_per_building",
                                "sky": "shifted_per_sky",
                                }, inplace=True)

id_df = pd.read_csv("data/raw/road_shoulder/gsv/metadata/gsv_metadata_cv_filtered.csv")
id_df["mly_id"] = id_df["mly_id"].astype(int).astype(str)

merged = pd.merge(id_df, ground_truth, left_on="mly_id", right_on="filename_key")
print(sidewalk["filename_key"])
print(merged["filename_key"])
merged = pd.merge(merged, sidewalk, on = "filename_key")
# merged = pd.merge(merged, shifted_pano, left_on="panoid", right_on="filename_key")
# merged = pd.merge(merged, shifted_per, left_on="panoid", right_on="filename_key")

# # calculate MSE, MAE, R^2, and Pearson correlation between ground_truth_vegetation and predicted_vegetation
# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# from scipy.stats import pearsonr

# df_list = []
# for variable in ["vegetation", "building", "sky"]:
#     mse_pano = mean_squared_error(merged[f"ground_truth_{variable}"], merged[f"shifted_pano_{variable}"])
#     mae_pano = mean_absolute_error(merged[f"ground_truth_{variable}"], merged[f"shifted_pano_{variable}"])
#     r2_pano = r2_score(merged[f"ground_truth_{variable}"], merged[f"shifted_pano_{variable}"])
#     pearson_pano = pearsonr(merged[f"ground_truth_{variable}"], merged[f"shifted_pano_{variable}"])[0]
    
#     mse_per = mean_squared_error(merged[f"ground_truth_{variable}"], merged[f"shifted_per_{variable}"])
#     mae_per = mean_absolute_error(merged[f"ground_truth_{variable}"], merged[f"shifted_per_{variable}"])
#     r2_per = r2_score(merged[f"ground_truth_{variable}"], merged[f"shifted_per_{variable}"])
#     pearson_per = pearsonr(merged[f"ground_truth_{variable}"], merged[f"shifted_per_{variable}"])[0]
    
#     mse_pano_sidewalk = mean_squared_error(merged[f"ground_truth_sidewalk_{variable}"], merged[f"shifted_pano_{variable}"])
#     mae_pano_sidewalk = mean_absolute_error(merged[f"ground_truth_sidewalk_{variable}"], merged[f"shifted_pano_{variable}"])
#     r2_pano_sidewalk = r2_score(merged[f"ground_truth_sidewalk_{variable}"], merged[f"shifted_pano_{variable}"])
#     pearson_pano_sidewalk = pearsonr(merged[f"ground_truth_sidewalk_{variable}"], merged[f"shifted_pano_{variable}"])[0]
    
#     mse_per_sidewalk = mean_squared_error(merged[f"ground_truth_sidewalk_{variable}"], merged[f"shifted_per_{variable}"])
#     mae_per_sidewalk = mean_absolute_error(merged[f"ground_truth_sidewalk_{variable}"], merged[f"shifted_per_{variable}"])
#     r2_per_sidewalk = r2_score(merged[f"ground_truth_sidewalk_{variable}"], merged[f"shifted_per_{variable}"])
#     pearson_per_sidewalk = pearsonr(merged[f"ground_truth_sidewalk_{variable}"], merged[f"shifted_per_{variable}"])[0]
    
#     df = pd.DataFrame({
#         "mse": [mse_pano, mse_per, mse_pano_sidewalk, mse_per_sidewalk],
#         "mae": [mae_pano, mae_per, mae_pano_sidewalk, mae_per_sidewalk],
#         "r2": [r2_pano, r2_per, r2_pano_sidewalk, r2_per_sidewalk],
#         "pearson": [pearson_pano, pearson_per, pearson_pano_sidewalk, pearson_per_sidewalk],
#         "variable": [variable, variable, variable, variable],
#         "method": ["pano", "per", "pano_sidewalk", "per_sidewalk"]
#     })
#     df_list.append(df)
    
# df = pd.concat(df_list)
# df.to_csv("data/processed/traditional_cv/result.csv")

0      1126770208003040
1      1202773767001481
2       735575394811200
3       555282096627593
4       561111252645669
             ...       
150     596389005647341
151     498517415786240
152     861789291815851
153     862021441740450
154     935332707381955
Name: filename_key, Length: 155, dtype: object
0       102982035841478
1       103829669084290
2       105888675509658
3       106102425482481
4       108175612012831
             ...       
648    5763097263735638
649    5882061678493376
650    5899699310040876
651    5927870303892543
652    5933390910023492
Name: filename_key, Length: 653, dtype: object
